# emoji 크롤링 및 파싱

In [222]:
# 파일이름
resultfilename='20230216_emoji_name_dict.csv'

## 사용법
### 1. unicode 이모지 오픈소스 xml 크롤링
- emoji unicode 컨소시움 (https://github.com/unicode-org)에서 cldr 페이지로 접근한다.
- 참고 프로젝트: 유니코드 콘소시움 (https://github.com/unicode-org)
- 언어별로 xml로 정리되어있는 원본페이지를 찾고, xml import를 한다.
- ".//annotation"위치에 있는 모든 데이터를 변환한다.
    - 도큐먼트 설명
        - //annotation[@cp] : emoji (key)
        - //annotation : name (value)
        - //annotation[@type] : TTS타입인지 아닌지 (condition)
- type이 tts인 도큐먼트의 emoji(key) : name(value) 쌍을 구한다.
### 2. 1번의 방법 * 한/영/프/스
1. 한: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml
1. 영: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/en.xml
1. 프: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/fr.xml
1. 스: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/es.xml

### 3. csv export
- 한/영/프/스 도큐먼트를 한 파일로 통합하여 csv export
- csv 설명
    - emoji: emoji
    - tts_ko: 한국어 name
    - tts_en: 영어 name
    - tts_fr: 불어 name
    - tts_es: 스페인 name

# 코드
## 1. input 필드, 예시 데이터 추출

In [220]:
# 한국어 -> 영어 -> 불어 -> 스페인 순서
urls=["https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml",
      "https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/en.xml",
      "https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/fr.xml",
      "https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/es.xml"]
cols=['ko','en','fr','es']
#resultfilename='emoji_name_dict.csv'


import pandas as pd
from urllib.request import urlopen
import xml.etree.ElementTree as ET
with urlopen('https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml') as f:
    df = pd.read_xml(f, xpath=".//annotation")
df[df['type']=='tts'].reset_index(drop=True)

,cp,annotation,type
0,🏻,하얀 피부,tts
1,🏼,연한 갈색 피부,tts
2,🏽,갈색 피부,tts
3,🏾,진한 갈색 피부,tts
4,🏿,검은색 피부,tts
...,...,...,...
1492,🏴,펄럭이는 검은색 깃발,tts
1493,🏳,펄럭이는 흰색 깃발,tts
1494,🏳‍⚧,트랜스젠더 깃발,tts
1495,🏳‍🌈,무지개 깃발,tts


## 2. 언어별로 테이블 추출 및 concat
1. xml to DataFrame 변환
1. //annotation 도큐먼트만 추출, type=='tts'인 것들만
1. 언어별로 추출한 것을 concat
1. concat할때 cp열과 annotation열 (name) 합침

In [216]:
# concat 리스트 선언
concat_df=[]

# lang별로 도큐먼트 추출 및 concat
for i in range(len(urls)):
    with urlopen(urls[i]) as f:
        df = pd.read_xml(f, xpath=".//annotation")
    temp=df[df['type']=='tts'].reset_index(drop=True)
    #temp.rename(columns={'annotation':cols[i]}, inplace=True)
    
    # cp열, annotation열만 추출하여 리스트에 담기
    concat_df.append(temp[['cp','annotation']])

In [7]:
# concat 리스트의 데이터프레임들을 concat 실시
test=pd.concat(concat_df, axis=1, join='inner')
test=test.loc[:,~test.T.duplicated()]
# 언어별로 열 이름 지정
test.columns=['cp']+cols
# 결과 출력
test.to_csv(resultfilename, index=False)
print(test[:10])

  cp            ko                      en  \
0  🏻         하얀 피부         light skin tone   
1  🏼      연한 갈색 피부  medium-light skin tone   
2  🏽         갈색 피부        medium skin tone   
3  🏾      진한 갈색 피부   medium-dark skin tone   
4  🏿        검은색 피부          dark skin tone   
5  ⚧      트랜스젠더 기호      transgender symbol   
6  🤌       꼬집는 손가락         pinched fingers   
7  🥲  눈물 흘리며 웃는 얼굴  smiling face with tear   
8  🥷            닌자                   ninja   
9  🥸        변장한 얼굴          disguised face   

                                    fr                                   es  
0                          peau claire                   tono de piel claro  
1              peau moyennement claire             tono de piel claro medio  
2                 peau légèrement mate                   tono de piel medio  
3                            peau mate            tono de piel oscuro medio  
4                          peau foncée                  tono de piel oscuro  
5  symbole de la communau